# How to print tags and create templates

## Set up notebook

Do all imports.

In [ ]:
from pathlib import Path
import cv2
import numpy as np
from pupil_apriltags import Detector
import matplotlib.pyplot as plt
import json

Specify the location of the `apriltag-imgs` directory as well as of the directory to which you want to save images and templates.

In [ ]:
img_src_dir = Path('../../../apriltag-imgs')
img_dst_dir = Path('.')

Choose a tag family.

In [ ]:
# Tag family (a string)
tag_family = 'tag36h11'

# Tag image base name (a string) - must be consistent with name of tag family
tag_basename = 'tag36_11_'

## How to print one tag with known dimensions

Choose a tag id.

In [ ]:
tag_id = 0

Read the tag image.

In [ ]:
img_src = cv2.imread(str(Path(img_src_dir, tag_family, f'{tag_basename}{tag_id:05d}.png')), cv2.IMREAD_UNCHANGED)

Make a copy of the tag image to save and print. (This isn't necessary - we do it for consistency later.)

In [ ]:
img_dst = img_src.copy()

Look at the tag image.

In [ ]:
# Create a figure with one axis
fig, ax = plt.subplots(1, 1)

# Show the image
ax.imshow(img_dst)

# Get the width and height of the image
w_img_px = img_dst.shape[1]
h_img_px = img_dst.shape[0]

# Put a tick at each pixel
ax.set_xticks(range(0, w_img_px, 1))
ax.set_yticks(range(0, h_img_px, 1))

# Draw a grid to show where the pixels are
for i in range(w_img_px):
    ax.plot([i - 0.5, i - 0.5], [0 - 0.5, h_img_px - 0.5], '-', color='C1')
for i in range(h_img_px):
    ax.plot([0 - 0.5, w_img_px - 0.5], [i - 0.5, i - 0.5], '-', color='C1')

Specify the size of the tag in pixels.

In [ ]:
tag_size_px = 8

We will use [matplotlib](https://matplotlib.org/) to create a figure with an image of this tag and to save this figure as a `png` file that can be printed. To do so, we will need to specify the dimensions of the axes in the figure where the image will be shown. These dimensions have the form `[left, bottom, width, height]`, where units are fractions of the figure width and height.

Here is one way to compute axes dimensions that satisfy three properties:
* There is at least a quarter-inch margin around the axes so the tag image is not cropped when printed.
* The tag is as big as possible.
* The size of each pixel when printed is a whole number of millimeters. (As a consquence, the side length - i.e., the distance between corners - of each tag will also be a whole number of millimeters.)

After computing the axes dimensions, we print out the tag size in millimeters for reference.

In [ ]:
# Width and height of figure in inches
w_fig_in = 8.5
h_fig_in = 11.

# Minimum margin around tag image in inches when printed
margin_in = 0.25

# Maximum width and height of axes in inches
w_ax_in = w_fig_in - (2 * margin_in)
h_ax_in = h_fig_in - (2 * margin_in)

# Number of millimeters in one inch
mm_per_inch = 25.4

# Maximum width and height of each pixel in whole millimeters
w_px_mm = np.floor(mm_per_inch * w_ax_in / w_img_px)
h_px_mm = np.floor(mm_per_inch * h_ax_in / h_img_px)

# Actual size of each pixel in whole millimeters
px_per_mm = min(w_px_mm, h_px_mm)

# Actual tag size in whole millimeters
tag_size_mm = px_per_mm * tag_size_px

# Width and height of axes as a fraction of figure width and height
w_ax_frac = (px_per_mm * w_img_px / mm_per_inch) / w_fig_in
h_ax_frac = (px_per_mm * h_img_px / mm_per_inch) / h_fig_in

# Bounding box of axes
ax_rect = [
    (1. - w_ax_frac) / 2.,  # <-- left
    (1. - h_ax_frac) / 2.,  # <-- bottom
    w_ax_frac,              # <-- width
    h_ax_frac,              # <-- height
]

# Print tag size in whole millimeters
print(f'Tag size: {tag_size_mm:.0f} mm')

Create and save the figure.

In [ ]:
# Create figure
fig = plt.figure(
    figsize=[w_fig_in, h_fig_in],
    dpi=300,
    facecolor='w',
)

# Create axis
ax = fig.add_axes(
    rect=ax_rect,
    frame_on=False,
    xticks=[],
    yticks=[],
)

# Add image to axis
ax.imshow(img_dst)

# Filename of figure to save
img_dst_filename = str(Path(img_dst_dir, f'{tag_basename}{tag_id:05d}-image.png'))

# Save figure as png
fig.savefig(img_dst_filename)

Create a tag detector.

In [ ]:
tag_detector = Detector(
    families=tag_family,
    nthreads=1,
    quad_decimate=1.0,
    quad_sigma=0.0,
    refine_edges=1,
    decode_sharpening=0.,
    debug=0,
)

Read the image we just saved, as grayscale.

In [ ]:
img = cv2.imread(img_dst_filename, cv2.IMREAD_GRAYSCALE)

Detect tags in this image. There should be exactly one.

In [ ]:
# Detect tags
tag_detections = tag_detector.detect(
    img,
    estimate_tag_pose=False,
    camera_params=None,
    tag_size=None,
)

# Make sure exactly one tag was detected
assert(len(tag_detections) == 1)

Show the image again, annotated with the tag detection.

In [ ]:
# Create figure
fig, ax = plt.subplots(1, 1, figsize=[w_fig_in, h_fig_in])

# Show image
ax.imshow(img, cmap='gray')

# Annotate with reference frame
o = tag_detections[0].corners[0]
w = tag_detections[0].corners[1][0] - o[0]
ax.plot([o[0], o[0] + 0.25 * w], [o[1], o[1]], '-', color='C0', linewidth=6)
ax.plot([o[0], o[0]], [o[1], o[1] - 0.25 * w], '-', color='C0', linewidth=6)
ax.text(o[0] + 0.25 * w, o[1] + 0.01 * w, '$x_{tag}$', color='C0', fontsize=18, verticalalignment='top')
ax.text(o[0] - 0.01 * w, o[1] - 0.25 * w, '$y_{tag}$', color='C0', fontsize=18, horizontalalignment='right')

# Annotate with tag detections
for d in tag_detections:
    # Show tag id at center of tag
    ax.text(
        d.center[0],
        d.center[1],
        f'Tag ID = {d.tag_id}',
        color='C1',
        fontsize=48,
        horizontalalignment='center',
    )
    
    # Show a dot at each corner and label it with its index
    for i_c, c in enumerate(d.corners):
        ax.plot(c[0], c[1], '.', color='C1', markersize=24)
        ax.text(c[0] + 50, c[1] - 25, f'{i_c}', color='C1', fontsize=24)


Create and save a template. This template is a list. Each element of this list corresponds to a tag and specifies the tag ID and the $(x, y, z)$ coordinates of each corner in meters with respect to the "tag" reference frame shown in the image. In this case, there is only one tag, so the template is a list of length one.

In [ ]:
# Number of millimeters in one meter
mm_per_m = 1000.

# Create template
template = {
    'tag_family': tag_family,
    'tags': [
        {
            'tag_id': tag_id,
            'corners': ((tag_size_mm / mm_per_m) * np.array([
                [0., 0., 0.], # bottom left
                [1., 0., 0.], # bottom right
                [1., 1., 0.], # top right
                [0., 1., 0.], # top left
            ])).tolist(),
        }
    ]
}

# Specify filename of template
template_filename = str(Path(img_dst_dir, f'{tag_basename}{tag_id:05d}-template.json'))

# Save template
with open(template_filename, 'w') as f:
    json.dump(template, f, indent=4)

## How to print a grid of tags with known dimensions

Find the width and height of each square tag image (including the border).

In [ ]:
tag_img_size_px = img_src.shape[0]
assert(tag_img_size_px == img_src.shape[1])

Specify the number of rows and columns in the tag grid.

In [ ]:
number_of_rows = 5
number_of_cols = 8

Create a blank image for the tag grid.

In [ ]:
# Make all pixels white and opaque by default
img_dst = 255 * np.ones((
    number_of_rows * tag_img_size_px,
    number_of_cols * tag_img_size_px,
    4,
), dtype=np.uint8)

Add tag images to the tag grid.

In [ ]:
for row in range(number_of_rows):
    for col in range(number_of_cols):
        tag_id = (row * number_of_cols) + col
        img_src = cv2.imread(
            str(Path(img_src_dir, tag_family, f'{tag_basename}{tag_id:05d}.png')),
            cv2.IMREAD_UNCHANGED,
        )
        i_start = ((number_of_rows - (row + 1)) * tag_img_size_px)
        j_start = (col * tag_img_size_px)
        img_dst[
            i_start:(i_start + tag_img_size_px),
            j_start:(j_start + tag_img_size_px),
            :,
        ] = img_src.copy()

Look at the tag grid image.

In [ ]:
# Create a figure with one axis
fig, ax = plt.subplots(1, 1, figsize=(img_dst.shape[1] / 5, img_dst.shape[0] / 5))

# Show the tag grid image
ax.imshow(img_dst)

# Get the width and height of the image
w_img_px = img_dst.shape[1]
h_img_px = img_dst.shape[0]

# Put a tick at each pixel
ax.set_xticks(range(0, w_img_px, 1))
ax.set_yticks(range(0, h_img_px, 1))
ax.tick_params(axis='x', labelsize=8)
ax.tick_params(axis='y', labelsize=8)

# Draw a grid to show where the pixels are
for i in range(w_img_px):
    ax.plot([i - 0.5, i - 0.5], [0 - 0.5, h_img_px - 0.5], '-', color='C1')
for i in range(h_img_px):
    ax.plot([0 - 0.5, w_img_px - 0.5], [i - 0.5, i - 0.5], '-', color='C1')

Again, to create a figure with an image of this tag grid, we need to specify the dimensions of the axes in the figure where the image will be shown. We do this just like before, noting that a second consequence of our choice to make the size of each pixel a whole number of millimeters is that the distance between each tag will also be a whole number of millimeters.

In [ ]:
# Width and height of figure in inches
w_fig_in = 11.
h_fig_in = 8.5

# Maximum width and height of axes in inches
w_ax_in = w_fig_in - (2 * margin_in)
h_ax_in = h_fig_in - (2 * margin_in)

# Maximum width and height of each pixel in whole millimeters
w_px_mm = np.floor(mm_per_inch * w_ax_in / w_img_px)
h_px_mm = np.floor(mm_per_inch * h_ax_in / h_img_px)

# Actual size of each pixel in whole millimeters
px_per_mm = min(w_px_mm, h_px_mm)

# Actual tag size in whole millimeters
tag_size_mm = px_per_mm * tag_size_px

# Actual tag border size in whole millimeters
tag_border_mm = px_per_mm * ((tag_img_size_px - tag_size_px) / 2)

# Width and height of axes as a fraction of figure width and height
w_ax_frac = (px_per_mm * w_img_px / mm_per_inch) / w_fig_in
h_ax_frac = (px_per_mm * h_img_px / mm_per_inch) / h_fig_in

# Bounding box of axes
ax_rect = [
    (1. - w_ax_frac) / 2.,  # <-- left
    (1. - h_ax_frac) / 2.,  # <-- bottom
    w_ax_frac,              # <-- width
    h_ax_frac,              # <-- height
]

# Print tag size and tag border size in whole millimeters
print(f'Tag size: {tag_size_mm:.0f} mm')
print(f'Tag border size: {tag_border_mm:.0f} mm')

Create and save the figure.

In [ ]:
# Create figure
fig = plt.figure(
    figsize=[w_fig_in, h_fig_in],
    dpi=300,
    facecolor='w',
)

# Create axis
ax = fig.add_axes(
    rect=ax_rect,
    frame_on=False,
    xticks=[],
    yticks=[],
)

# Add tag image to axis
ax.imshow(img_dst)

# Filename of figure to save
img_dst_filename = str(Path(img_dst_dir, f'{tag_basename}grid_{number_of_rows}x{number_of_cols}-image.png'))

# Save figure as png
fig.savefig(img_dst_filename)

Read the image we just saved, as grayscale.

In [ ]:
img = cv2.imread(img_dst_filename, cv2.IMREAD_GRAYSCALE)

Detect tags in this image. There should be exactly fourty.

In [ ]:
# Detect tags
tag_detections = tag_detector.detect(
    img,
    estimate_tag_pose=False,
    camera_params=None,
    tag_size=None,
)

# Make sure exactly one tag was detected
assert(len(tag_detections) == number_of_cols * number_of_rows)

Show the image again, annotated with all tag detections.

In [ ]:
# Create figure
fig, ax = plt.subplots(1, 1, figsize=[w_fig_in, h_fig_in])

# Show image
ax.imshow(img, cmap='gray')

# Annotate with reference frame
o = tag_detections[0].corners[0]
w = tag_detections[0].corners[1][0] - o[0]
ax.plot([o[0], o[0] + 0.75 * w], [o[1], o[1]], '-', color='C0', linewidth=4)
ax.plot([o[0], o[0]], [o[1], o[1] - 0.75 * w], '-', color='C0', linewidth=4)
ax.text(o[0] + 0.75 * w, o[1] + 0.05 * w, '$x_{tag}$', color='C0', fontsize=18, verticalalignment='top')
ax.text(o[0] - 0.05 * w, o[1] - 0.75 * w, '$y_{tag}$', color='C0', fontsize=18, horizontalalignment='right')

# Annotate with tag detections
for d in tag_detections:
    # Show tag id at center of tag
    ax.text(
        d.center[0],
        d.center[1],
        f'{d.tag_id}',
        color='C1',
        fontsize=24,
        weight='bold',
        verticalalignment='center',
        horizontalalignment='center',
    )
    
    # Show a dot at each corner
    for i_c, c in enumerate(d.corners):
        ax.plot(c[0], c[1], '.', color='C1', markersize=9)
        ax.text(c[0] + 8, c[1] - 8, f'{i_c}', color='C1', fontsize=9)


Create and save a template. This template is a list. Each element of this list corresponds to a tag and specifies the tag ID and the $(x, y, z)$ coordinates of each corner in meters with respect to the "tag" reference frame shown in the image.

In [ ]:
# Create a canonical set of corners
corners = (tag_size_mm / mm_per_m) * np.array([
    [0., 0., 0.], # bottom left
    [1., 0., 0.], # bottom right
    [1., 1., 0.], # top right
    [0., 1., 0.], # top left
])

# Create template
template = {
    'tag_family': tag_family,
    'tags': [],
}
for row in range(number_of_rows):
    for col in range(number_of_cols):
        tag_id = (row * number_of_cols) + col
        bottom_left = np.array([
            col * (tag_size_mm + 2 * tag_border_mm) / mm_per_m,
            row * (tag_size_mm + 2 * tag_border_mm) / mm_per_m,
            0.,
        ])
        template['tags'].append({
            'tag_id': tag_id,
            'corners': np.round((corners + bottom_left), decimals=3).tolist()
        })

# Specify filename of template
template_filename = str(Path(img_dst_dir, f'{tag_basename}grid_{number_of_rows}x{number_of_cols}-template.json'))

# Save template
with open(template_filename, 'w') as f:
    json.dump(template, f, indent=4)

Visualize the template to make sure it is correct.

In [ ]:
# Create figure
fig, ax = plt.subplots(1, 1, figsize=(2 * number_of_rows, 2 * number_of_cols))

# Create sets to hold ticks so grid-lines go through tag corners
xticks = set()
yticks = set()

# Iterate over all tags
for tag in template['tags']:
    # Get tag properties
    tag_id = tag['tag_id']
    corners = np.array(tag['corners'])
    center = np.mean(corners, 0)

    # Iterate over all corners
    for i_c, c in enumerate(corners):
        # Add corners to tick sets
        xticks.add(c[0])
        yticks.add(c[1])

        # Show and label corners
        ax.plot(c[0], c[1], '.', color='C1', markersize=6)
        ax.text(c[0] + 0.001, c[1] + 0.001, f'{i_c}', color='C1', fontsize=9)
    
        # Show tag id at center of tag
        ax.text(
            center[0],
            center[1],
            f'{tag_id}',
            color='C1',
            fontsize=16,
            weight='bold',
            verticalalignment='center',
            horizontalalignment='center',
        )

# Make everything look nice
ax.set_aspect('equal')
ax.grid()
ax.set_xticks(list(xticks))
ax.set_yticks(list(yticks))
ax.tick_params(axis='x', labelsize=9, labelrotation=90.)
ax.tick_params(axis='y', labelsize=9)